## Match searches with bookings

- For every search in the searches file, find out whether the search ended up in a booking or not (using the info in the bookings file). For instance, search and booking origin and destination should match. 

- For the bookings file, origin and destination are the columns dep_port and arr_port, respectively. 

- Generate a CSV file with the search data, and an additional field, containing 1 if the search ended up in a booking, and 0 otherwise.



## Plan

most simple:
match
Search : [search_date, O&D] 
Booking: [Activity_date, O&D]

if there is one booking for a given O&D done at the same day as the search for a given O&D, all searches of the day for a given 0&D might have resulted from the same source and will be set with 1

1) create the 2 DF
2) search left join booking

Questions:
1) do we have duplicated lines in searhes/bookings
2) any white space 

### Booking

In [13]:
all_chunks=pd.DataFrame()
bc=pd.read_csv('bookings.csv.bz2',sep='^', chunksize=1000000, low_memory=False)

for i,chunk in enumerate(bc):
    all_chunks=all_chunks.append(chunk)
    all_chunks=all_chunks.drop_duplicates()
    print (i, len(all_chunks))

(0, 1000000)
(1, 1000000)
(2, 1000000)
(3, 1000000)
(4, 1000000)
(5, 1999993)
(6, 1999993)
(7, 1999993)
(8, 1999993)
(9, 1999993)
(10, 1999993)


In [18]:
all_chunks.columns.tolist()

['act_date           ',
 'source',
 'pos_ctry',
 'pos_iata',
 'pos_oid  ',
 'rloc          ',
 'cre_date           ',
 'duration',
 'distance',
 'dep_port',
 'dep_city',
 'dep_ctry',
 'arr_port',
 'arr_city',
 'arr_ctry',
 'lst_port',
 'lst_city',
 'lst_ctry',
 'brd_port',
 'brd_city',
 'brd_ctry',
 'off_port',
 'off_city',
 'off_ctry',
 'mkt_port',
 'mkt_city',
 'mkt_ctry',
 'intl',
 'route          ',
 'carrier',
 'bkg_class',
 'cab_class',
 'brd_time           ',
 'off_time           ',
 'pax',
 'year',
 'month',
 'oid      ']

In [23]:
all_chunks.columns=all_chunks.columns.map(lambda x:str(x).strip())

In [28]:
#list(all_chunks.columns)

In [26]:
all_chunks.columns.tolist()

['act_date',
 'source',
 'pos_ctry',
 'pos_iata',
 'pos_oid',
 'rloc',
 'cre_date',
 'duration',
 'distance',
 'dep_port',
 'dep_city',
 'dep_ctry',
 'arr_port',
 'arr_city',
 'arr_ctry',
 'lst_port',
 'lst_city',
 'lst_ctry',
 'brd_port',
 'brd_city',
 'brd_ctry',
 'off_port',
 'off_city',
 'off_ctry',
 'mkt_port',
 'mkt_city',
 'mkt_ctry',
 'intl',
 'route',
 'carrier',
 'bkg_class',
 'cab_class',
 'brd_time',
 'off_time',
 'pax',
 'year',
 'month',
 'oid']

In [60]:
b=all_chunks[['cre_date','act_date', 'dep_port', 'arr_port']].copy()
#si es un round trip lo vamos a tener 2 veces

In [61]:
b.head()

,cre_date,act_date,dep_port,arr_port
0,2013-02-22 00:00:00,2013-03-05 00:00:00,ZRH,LHR
1,2013-03-26 00:00:00,2013-03-26 00:00:00,SAL,CLT
2,2013-03-26 00:00:00,2013-03-26 00:00:00,SAL,CLT
3,2013-03-26 00:00:00,2013-03-26 00:00:00,AKL,SVO
4,2013-03-26 00:00:00,2013-03-26 00:00:00,AKL,SVO


###  Search

In [150]:
import pandas as pd
all_chunks=pd.DataFrame()
bc=pd.read_csv('searches.clean.csv.bz2',sep='^', chunksize=1000000)

for i,chunk in enumerate(bc):
    all_chunks=all_chunks.append(chunk)
    all_chunks=all_chunks.drop_duplicates()
    print (i, len(all_chunks))
    
    
    

(0, 363343)
(1, 389521)
(2, 399321)
(3, 415079)
(4, 415079)
(5, 415079)
(6, 415079)
(7, 421663)
(8, 421663)
(9, 421663)
(10, 421871)
(11, 421871)
(12, 421871)
(13, 421871)
(14, 421975)
(15, 421975)
(16, 421975)
(17, 422183)
(18, 422183)
(19, 422183)
(20, 422184)


In [152]:
s=all_chunks[['Date', 'Origin','Destination']]

In [153]:
s.head()

,Date,Origin,Destination
0,2013-01-01,TXL,AUH
1,2013-01-01,ATH,MIL
2,2013-01-01,ICT,SFO
3,2013-01-01,RNB,ARN
4,2013-01-01,OSL,MAD


### Modifing the columns

In [63]:
print "size of Book:\t%d \nsize of Search:\t%d " % (len(b), len(s))

size of Book:	1999993 
size of Search:	422183 


In [64]:
b=b.dropna()

In [67]:
b['act_date']=pd.to_datetime(b['act_date'],errors='raise')

In [68]:
b.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1999993 entries, 0 to 999999
Data columns (total 4 columns):
cre_date    object
act_date    datetime64[ns]
dep_port    object
arr_port    object
dtypes: datetime64[ns](1), object(3)
memory usage: 76.3+ MB


In [69]:
b.drop('cre_date', axis=1, inplace=True)
b.head()

,act_date,dep_port,arr_port
0,2013-03-05,ZRH,LHR
1,2013-03-26,SAL,CLT
2,2013-03-26,SAL,CLT
3,2013-03-26,AKL,SVO
4,2013-03-26,AKL,SVO


In [70]:
list(b.dep_port.head())

['ZRH     ', 'SAL     ', 'SAL     ', 'AKL     ', 'AKL     ']

In [71]:
b['dep_port']=b['dep_port'].map(lambda x: x.strip())
b['arr_port']=b['arr_port'].map(lambda x: x.strip())

In [72]:
list(b.dep_port.head())

['ZRH', 'SAL', 'SAL', 'AKL', 'AKL']

In [65]:
print "size of Book:\t%d \nsize of Search:\t%d " % (len(b), len(s))

size of Book:	1999993 
size of Search:	422183 


In [155]:
s=s.dropna()
s['Date'] = pd.to_datetime(s['Date'],errors='raise')

### The real code starts from here

In [73]:
b=b.reset_index()
b.head()

,index,act_date,dep_port,arr_port
0,0,2013-03-05,ZRH,LHR
1,1,2013-03-26,SAL,CLT
2,2,2013-03-26,SAL,CLT
3,3,2013-03-26,AKL,SVO
4,4,2013-03-26,AKL,SVO


In [74]:
b.rename(columns={'index':'book ID'}, inplace=True)
b.head()

,book ID,act_date,dep_port,arr_port
0,0,2013-03-05,ZRH,LHR
1,1,2013-03-26,SAL,CLT
2,2,2013-03-26,SAL,CLT
3,3,2013-03-26,AKL,SVO
4,4,2013-03-26,AKL,SVO


In [75]:
s=s.reset_index()
s.rename(columns={'index':'search ID'}, inplace=True)
s.head()

In [76]:
s.head()

,search ID,Date,Origin,Destination
0,0,2013-01-01,TXL,AUH
1,1,2013-01-01,ATH,MIL
2,2,2013-01-01,ICT,SFO
3,3,2013-01-01,RNB,ARN
4,4,2013-01-01,OSL,MAD


In [78]:
s.sort_values(by=['Origin','Destination', 'Date'],inplace=True)
s.head(10)

,search ID,Date,Origin,Destination
317322,317323,2013-11-14,240,INR
49789,49789,2013-02-19,AAC,CAI
170497,170497,2013-06-20,AAE,ALG
141836,141836,2013-05-22,AAE,MRS
231815,231816,2013-08-20,AAE,NCE
278500,278501,2013-10-06,AAE,ORY
181303,181303,2013-07-01,AAE,PAR
234611,234612,2013-08-23,AAE,PAR
99103,99103,2013-04-10,AAL,AGP
169509,169509,2013-06-19,AAL,AGP


In [98]:
int(s.ix[317322]['Origin'])

240

In [204]:
def check_airports (airport):
    try:
        int(airport)
        print "%s is not airport"%(airport)
        return False
    except:
        return True

In [199]:
check_airports('123')

123 is not airport


False

In [205]:
s.head().Origin.map(check_airports)

0    True
1    True
2    True
3    True
4    True
Name: Origin, dtype: bool

In [206]:
s_test=s.head()

s_test2 = s[s.Origin.map(check_airports)]
s_test2

240 is not airport


,Date,Origin,Destination
0,2013-01-01,TXL,AUH
1,2013-01-01,ATH,MIL
2,2013-01-01,ICT,SFO
3,2013-01-01,RNB,ARN
4,2013-01-01,OSL,MAD
5,2013-01-01,IAH,BLR
6,2013-01-01,CPH,PAR
7,2013-01-01,PAR,DUB
8,2013-01-01,DUS,ACE
9,2013-01-01,FRA,BGW


In [106]:
s = s[s.Origin.map(check_airports)]
s = s[s.Destination.map(check_airports)]
len(s)

422182

### izbaceni brojevi

In [111]:
b.sort_values(['dep_port','arr_port', 'act_date'],inplace=True)

In [112]:
print s.head(10)
print b.head(10)

        search ID       Date Origin Destination
49789       49789 2013-02-19    AAC         CAI
170497     170497 2013-06-20    AAE         ALG
141836     141836 2013-05-22    AAE         MRS
231815     231816 2013-08-20    AAE         NCE
278500     278501 2013-10-06    AAE         ORY
181303     181303 2013-07-01    AAE         PAR
234611     234612 2013-08-23    AAE         PAR
99103       99103 2013-04-10    AAL         AGP
169509     169509 2013-06-19    AAL         AGP
184042     184042 2013-07-04    AAL         AGP
         book ID   act_date dep_port arr_port
387078    387078 2013-05-29      AAB      ROV
387079    387079 2013-05-29      AAB      ROV
1387081   387088 2013-05-29      AAB      ROV
1387082   387089 2013-05-29      AAB      ROV
572522    572522 2013-08-24      AAB      ROV
572523    572523 2013-08-24      AAB      ROV
1572525   572532 2013-08-24      AAB      ROV
1572526   572533 2013-08-24      AAB      ROV
165186    165186 2013-01-07      AAE      ALG
1165189   16

In [195]:
b_gr=b.groupby(['dep_port','arr_port', 'act_date']).count() .reset_index().sort_values(
    ['dep_port','arr_port', 'act_date'])

In [129]:
b_gr.rename(columns={'book ID':'Num of Bookings'}, inplace=True)
print len(b_gr)
b_gr.head()

466228


,dep_port,arr_port,act_date,Num of Bookings
0,AAB,ROV,2013-05-29,4
1,AAB,ROV,2013-08-24,4
2,AAE,ALG,2013-01-07,2
3,AAE,ALG,2013-01-08,4
4,AAE,ALG,2013-01-09,8


In [128]:
s.head()

,search ID,Date,Origin,Destination
49789,49789,2013-02-19,AAC,CAI
170497,170497,2013-06-20,AAE,ALG
141836,141836,2013-05-22,AAE,MRS
231815,231816,2013-08-20,AAE,NCE
278500,278501,2013-10-06,AAE,ORY


In [130]:
pd.merge?

In [156]:
s_b=s.merge(b_gr, 
            how='left', 
            left_on=['Date','Origin','Destination'], 
            right_on=['act_date', 'dep_port','arr_port'])

In [157]:
s_b[s_b['Num of Bookings']>0].head()

,Date,Origin,Destination,dep_port,arr_port,act_date,Num of Bookings
23,2013-01-01,DUS,IST,DUS,IST,2013-01-01,2
27,2013-01-01,RUH,JED,RUH,JED,2013-01-01,26
40,2013-01-01,DMM,MNL,DMM,MNL,2013-01-01,2
59,2013-01-01,ATL,MIA,ATL,MIA,2013-01-01,4
94,2013-01-01,DXB,KUL,DXB,KUL,2013-01-01,2


In [158]:
len(s_b)

422183

In [142]:
s.head()

,search ID,Date,Origin,Destination
49789,49789,2013-02-19,AAC,CAI
170497,170497,2013-06-20,AAE,ALG
141836,141836,2013-05-22,AAE,MRS
231815,231816,2013-08-20,AAE,NCE
278500,278501,2013-10-06,AAE,ORY


In [159]:
all_chunks.head()


,Date,Time,TxnCode,OfficeID,Country,Origin,Destination,RoundTrip,NbSegments,Seg1Departure,...,Seg6Arrival,Seg6Date,Seg6Carrier,Seg6BookingCode,From,IsPublishedForNeg,IsFromInternet,IsFromVista,TerminalID,InternetOffice
0,2013-01-01,20:25:57,MPT,624d8c3ac0b3a7ca03e3c167e0f48327,DE,TXL,AUH,1,2,TXL,...,NaN,NaN,NaN,NaN,1ASIWS,0,0,0,d41d8cd98f00b204e9800998ecf8427e,FRA
1,2013-01-01,10:15:33,MPT,b0af35b31588dc4ab06d5cf2986e8e02,MD,ATH,MIL,0,1,ATH,...,NaN,NaN,NaN,NaN,1ASIWS,0,0,0,d41d8cd98f00b204e9800998ecf8427e,KIV
2,2013-01-01,18:04:49,MPT,3561a60621de06ab1badc8ca55699ef3,US,ICT,SFO,1,2,ICT,...,NaN,NaN,NaN,NaN,1ASIWS,0,0,0,d41d8cd98f00b204e9800998ecf8427e,NYC
3,2013-01-01,17:42:40,FXP,1864e5e8013d9414150e91d26b6a558b,SE,RNB,ARN,0,1,RNB,...,NaN,NaN,NaN,NaN,1ASI,0,0,0,d41d8cd98f00b204e9800998ecf8427e,STO
4,2013-01-01,17:48:29,MPT,1ec336348f44207d2e0027dc3a68c118,NO,OSL,MAD,1,2,OSL,...,NaN,NaN,NaN,NaN,1ASIWS,0,0,0,d41d8cd98f00b204e9800998ecf8427e,OSL


In [160]:
len(all_chunks)

422184

In [190]:
result=pd.concat([all_chunks, s_b['Num of Bookings']])
result.rename(columns={0:'Booked'}, inplace=True)
result.head()

,Booked,Country,Date,Destination,From,InternetOffice,IsFromInternet,IsFromVista,IsPublishedForNeg,NbSegments,...,Seg5Date,Seg5Departure,Seg6Arrival,Seg6BookingCode,Seg6Carrier,Seg6Date,Seg6Departure,TerminalID,Time,TxnCode
0,NaN,DE,2013-01-01,AUH,1ASIWS,FRA,0,0,0,2,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,d41d8cd98f00b204e9800998ecf8427e,20:25:57,MPT
1,NaN,MD,2013-01-01,MIL,1ASIWS,KIV,0,0,0,1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,d41d8cd98f00b204e9800998ecf8427e,10:15:33,MPT
2,NaN,US,2013-01-01,SFO,1ASIWS,NYC,0,0,0,2,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,d41d8cd98f00b204e9800998ecf8427e,18:04:49,MPT
3,NaN,SE,2013-01-01,ARN,1ASI,STO,0,0,0,1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,d41d8cd98f00b204e9800998ecf8427e,17:42:40,FXP
4,NaN,NO,2013-01-01,MAD,1ASIWS,OSL,0,0,0,2,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,d41d8cd98f00b204e9800998ecf8427e,17:48:29,MPT


In [193]:
result.Booked.fillna(value=0, inplace=True)
result.head()

,Booked,Country,Date,Destination,From,InternetOffice,IsFromInternet,IsFromVista,IsPublishedForNeg,NbSegments,...,Seg5Date,Seg5Departure,Seg6Arrival,Seg6BookingCode,Seg6Carrier,Seg6Date,Seg6Departure,TerminalID,Time,TxnCode
0,0,DE,2013-01-01,AUH,1ASIWS,FRA,0,0,0,2,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,d41d8cd98f00b204e9800998ecf8427e,20:25:57,MPT
1,0,MD,2013-01-01,MIL,1ASIWS,KIV,0,0,0,1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,d41d8cd98f00b204e9800998ecf8427e,10:15:33,MPT
2,0,US,2013-01-01,SFO,1ASIWS,NYC,0,0,0,2,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,d41d8cd98f00b204e9800998ecf8427e,18:04:49,MPT
3,0,SE,2013-01-01,ARN,1ASI,STO,0,0,0,1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,d41d8cd98f00b204e9800998ecf8427e,17:42:40,FXP
4,0,NO,2013-01-01,MAD,1ASIWS,OSL,0,0,0,2,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,d41d8cd98f00b204e9800998ecf8427e,17:48:29,MPT
